In [1]:
!pip install Flask pandas numpy scikit-learn nltk spacy joblib openpyxl

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 989.2 kB/s eta 0:00:11
   -- ------------------------------------- 0.8/11.1 MB 1.1 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/11.1 MB 1.1 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/11.1 MB 1.1 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/11.1 MB 972.7 kB/s eta 0:00:11
   ---- ----------------------------------- 1.3/11.1 MB 972.7 kB/s eta 0:00:11
   ----- ---------------------------------- 1.6/11.1 MB 883.2 kB/s eta 0:00:11
   ----- ---------------------------------- 1.6/11.1 MB 883.2 kB/s eta 0:00:11
   ------ --------------------------------- 1.8/11.1 MB 818.3 kB/s eta 0:00:12
   ------- ----

In [2]:
!pip install pandas python-docx

## Chuẩn Bị Bộ Dữ Liệu

In [1]:
# Import các thư viện
import pandas as pd
import os
import PyPDF2
import re
import spacy
import joblib
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Tải cài đặt NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tgdd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
# Tải mô hình SpaCy
nlp = spacy.load("en_core_web_sm")

In [4]:
# Đường dẫn đến file
file_path = "D:\BaiDoAnChuyenNganh3\Automated-Resume-Ranking-System-main\csvfiles\crawlcv\DataFinal\GeneralCV_Example.xlsx"

In [5]:
# Kiểm tra xem file có tồn tại không
if not os.path.exists(file_path):
    print(f"Lỗi: File không tồn tại tại đường dẫn: {file_path}")
    print("Vui lòng kiểm tra lại đường dẫn hoặc tên file.")
    print("Gợi ý: Đảm bảo file là '.xlsx' hoặc thử '.csv' nếu đó là định dạng đúng.")
    raise FileNotFoundError(f"File không tồn tại: {file_path}")

In [6]:
# Tải dataset GeneralCV_Example.xlsx
df = pd.read_excel(file_path)

In [7]:
# In 5 dòng đầu để kiểm tra
print(df.head())

   ID                                         Resume_str  \
0   1           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1   2           HR SPECIALIST, US HR OPERATIONS      ...   
2   3           HR DIRECTOR       Summary      Over 2...   
3   4           HR SPECIALIST       Summary    Dedica...   
4   5           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


### Tiền Xử Lý Dữ Liêu

In [8]:
# Kiểm tra giá trị thiếu trong cột Resume_str
print(f"Số giá trị thiếu trong Resume_str: {df['Resume_str'].isna().sum()}")

Số giá trị thiếu trong Resume_str: 1


In [9]:
# Xóa các dòng có giá trị thiếu trong Resume_str
df = df.dropna(subset=['Resume_str']).reset_index(drop=True)

In [10]:
# Đặt lại cột ID (tạo ID mới từ 1 đến số dòng)
df['ID'] = range(1, len(df) + 1)

In [11]:
# In số dòng sau khi xóa giá trị thiếu
print(f"Số dòng sau khi xóa giá trị thiếu: {len(df)}")

Số dòng sau khi xóa giá trị thiếu: 4171


In [12]:
df.head(10)

,ID,Resume_str,Resume_html,Category
0,1,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,2,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,3,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,4,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,5,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
5,6,HR GENERALIST Summary Dedic...,"<div class=""fontsize fontface vmargins hmargin...",HR
6,7,HR MANAGER Summary HUMAN RES...,"<div class=""fontsize fontface vmargins hmargin...",HR
7,8,HR MANAGER Professional Summa...,"<div class=""fontsize fontface vmargins hmargin...",HR
8,9,HR SPECIALIST Summary Posses...,"<div class=""fontsize fontface vmargins hmargin...",HR
9,10,HR CLERK Summary Translates ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [13]:
# Chức năng trích xuất văn bản từ PDF
def extract_text_from_resume(resume_path):
    text = ""
    try:
        with open(resume_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Lỗi khi trích xuất văn bản từ {resume_path}: {e}")
    return text.strip()

In [14]:
# Chức năng làm sạch văn bản
def clean_text(text):
    # Chuyển về chữ thường
    text = text.lower()
    # Chỉ giữ lại chữ cái (loại bỏ số, dấu câu, ký tự đặc biệt)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [15]:
# Tiền xử lý văn bản
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha and token.text not in stop_words]
    return ' '.join(tokens)

In [16]:
# Xử lý dữ liệu văn bản 
df["Processed_Resume_str"] = df["Resume_str"].apply(lambda x: preprocess_text(clean_text(x)))

In [21]:
df.head(5)

,ID,Resume_str,Resume_html,Category,Processed_Resume_str
0,1,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr administratormarkete associate hr administr...
1,2,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist hr operation summary versatile m...
2,3,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr director summary year experience recruiting...
3,4,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist summary dedicate driven dynamic ...
4,5,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr manager skill highlight hr skill hr departm...


In [22]:
# Kiểm tra và xóa các dòng trùng lặp trong cột Processed_Resume_str
print(f"Số dòng trước khi xóa trùng lặp: {len(df)}")
duplicates = df[df['Processed_Resume_str'].duplicated(keep=False)]
if not duplicates.empty:
    print(f"Số dòng trùng lặp trong Processed_Resume_str: {len(duplicates)}")
    print("Các dòng trùng lặp:")
    print(duplicates[['ID', 'Processed_Resume_str']])
else:
    print("Không có dòng trùng lặp trong Processed_Resume_str.")

Số dòng trước khi xóa trùng lặp: 4171
Số dòng trùng lặp trong Processed_Resume_str: 66
Các dòng trùng lặp:
        ID                               Processed_Resume_str
1489  1490  finance officer professional summary attain fu...
1508  1509  finance officer professional summary attain fu...
2443  2444  storekeeper ii professional summary purpose do...
2482  2483  storekeeper ii professional summary purpose do...
3128  3129  brittany finn fake street city state zip code ...
...    ...                                                ...
4119  4120  azim khalid fake street city state zip code e ...
4122  4123  kelly zuniga fake street city state zip code e...
4153  4154  jamal wilson fake street city state zip code e...
4159  4160  jennifer ikeda fake street city state zip code...
4164  4165  geena meadow fake street city state zip code e...

[66 rows x 2 columns]


In [23]:
# Xóa các dòng trùng lặp, giữ lại dòng đầu tiên
df = df.drop_duplicates(subset=['Processed_Resume_str'], keep='first').reset_index(drop=True)
print(f"Số dòng sau khi xóa trùng lặp: {len(df)}")

Số dòng sau khi xóa trùng lặp: 4138


In [24]:
# Đặt lại cột ID (tạo ID mới từ 1 đến số dòng)
df['ID'] = range(1, len(df) + 1)

In [41]:
# Chuyển đổi văn bản thành TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["Processed_Resume_str"])
y = df["Category"]  # Assuming there is a 'Category' column

In [42]:
# Chia tập dữ liệu huấn luyện, kiểm tra 80 20 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
# Đào tạo mô hình hồi quy logistic
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
log_accuracy = accuracy_score(y_test, log_model.predict(X_test))
print(f"Logistic Regression Độ Chính Xác: {log_accuracy:.2f}")
print(classification_report(y_test, log_model.predict(X_test)))

Logistic Regression Độ Chính Xác: 0.64
                        precision    recall  f1-score   support

            ACCOUNTANT       0.81      0.86      0.83        29
              ADVOCATE       0.57      0.57      0.57        30
           AGRICULTURE       0.50      0.12      0.20         8
               APPAREL       0.56      0.45      0.50        20
                  ARTS       0.12      0.11      0.12        18
            AUTOMOBILE       0.00      0.00      0.00         6
              AVIATION       0.69      0.86      0.77        21
               BANKING       0.75      0.65      0.70        23
                   BPO       0.00      0.00      0.00         2
  BUSINESS-DEVELOPMENT       0.80      0.59      0.68        27
                  CHEF       0.85      0.71      0.77        24
          CONSTRUCTION       0.86      0.74      0.79        34
            CONSULTANT       0.40      0.30      0.34        20
              DESIGNER       0.71      0.79      0.75        19


c:\Users\tgdd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tgdd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tgdd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [ ]:
# Đào tạo mô hình Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_accuracy = accuracy_score(y_test, rf_model.predict(X_test))
print(f"Random Forest Độ Chính Xác: {rf_accuracy:.2f}")
print(classification_report(y_test, rf_model.predict(X_test)))

In [ ]:
# Lưu mô hình
joblib.dump(log_model, "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/Contacts/logistic_model.pkl")
joblib.dump(rf_model, "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/Contacts/random_forest_model.pkl")
joblib.dump(vectorizer, "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/Contacts/tfidf_vectorizer.pkl")

print("Mô hình đã được lưu thành công!")

NameError: name 'rf_model' is not defined